# Import

In [7]:
import requests
from bs4 import BeautifulSoup

## Webページをダウンロード

In [11]:
url = 'https://www.seshop.com/'
r = requests.get(url)
'''
各ステータスの確認

print(type(r))
print(r.status_code)
'''

# htmlのソースを取得
text = r.text
for line in text.split('\n'):
    if '<title>' in line or '<h2>' in line:
        # 余分な空白を削除し確認しやすくするためにstrip()を使用
        print(line.strip())

<title>SEshop｜ 翔泳社の本・電子書籍通販サイト</title>
<h2>予約ランキング</h2>
<h2>書籍ランキング <span class="pull-right"><a href="/ranking/1"><span class="glyphicon glyphicon-chevron-right"></span> もっと見る</a></span></h2>
<h2>電子書籍ランキング<span class="pull-right"><a href="/ranking/327/"><span class="glyphicon glyphicon-chevron-right"></span> もっと見る</a></span></h2>
<h2>新刊案内</h2>
<h2>申し込み受付中のセミナー</h2>
<h2>商品カテゴリー</h2>
<h2>人気商品ランキング</h2>
<h2>キャンペーン・特集</h2>


## Webページから要素を抜き出す

In [15]:
# htmlを解析したBeautifulSourpオブジェクトを生成
soup = BeautifulSoup(text,'html.parser')
# <title>タグの表示
print(soup.title)
# <h2>タグの表示
print(soup.h2)

# <h2>タグをすべて取得
h2_tags = soup.find_all('h2')

for h2 in h2_tags:
    # 各<h2>タグ内の<a>タグを探す
    a_tag = h2.find('a')
    # print(a_tag)
    # <a>タグが存在し、href属性を持っている場合、そのhref属性を表示
    if a_tag and a_tag.has_attr('href'):
        print(a_tag['href'])

<title>SEshop｜ 翔泳社の本・電子書籍通販サイト</title>
<h2>予約ランキング</h2>
/ranking/1
/ranking/327/
